In [1]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/jcmor/desktop/Ironhack/ironhack_service_account_big_query.json"
from google.cloud import bigquery

# Instantiates a client for BigQuery Service
client = bigquery.Client()

In [2]:
query1 = '''
SELECT   
    titleauthor.au_id AS AUTHOR_ID,
    titleauthor.title_id AS TITLE_ID,
    titles.price * sales.qty * titles.royalty / 100 * titleauthor.royaltyper / 100 AS ROYALTIES,
FROM
    `ironhack-data-analytics-265219.publications.titleauthor` titleauthor
JOIN
    `ironhack-data-analytics-265219.publications.sales` sales
ON
    sales.title_id = titleauthor.title_id
JOIN
    `ironhack-data-analytics-265219.publications.titles` titles
ON
    titleauthor.title_id = titles.title_id
'''

In [3]:
query_job = client.query(query=query1)
df=query_job.to_dataframe()
df

,AUTHOR_ID,TITLE_ID,ROYALTIES
0,899-46-2035,PS2091,1.971
1,899-46-2035,PS2091,49.275
2,899-46-2035,PS2091,6.570
3,899-46-2035,PS2091,13.140
4,846-92-7186,PC8888,50.000
5,213-46-8915,BU1032,3.998
6,213-46-8915,BU1032,7.996
7,267-41-2394,BU1111,11.950
8,267-41-2394,TC7777,8.994
9,899-46-2035,MC3021,4.485


In [4]:
query2 = '''
SELECT   
    titleauthor.au_id AS AUTHOR_ID,
    titleauthor.title_id AS TITLE_ID,
    SUM(titles.price * sales.qty * titles.royalty / 100 * titleauthor.royaltyper / 100) AS ROYALTIES,
FROM
    `ironhack-data-analytics-265219.publications.titleauthor` titleauthor
JOIN
    `ironhack-data-analytics-265219.publications.sales` sales
ON
    sales.title_id = titleauthor.title_id
JOIN
    `ironhack-data-analytics-265219.publications.titles` titles
ON
    titleauthor.title_id = titles.title_id
GROUP BY
    1,2
'''

In [6]:
query_job = client.query(query=query2)
df=query_job.to_dataframe()
df

,AUTHOR_ID,TITLE_ID,ROYALTIES
0,899-46-2035,PS2091,70.956
1,846-92-7186,PC8888,50.000
2,213-46-8915,BU1032,11.994
3,267-41-2394,BU1111,11.950
4,267-41-2394,TC7777,8.994
5,899-46-2035,MC3021,7.176
6,724-80-9391,PS1372,10.795
7,756-30-7391,PS1372,32.385
8,722-51-5454,MC3021,21.528
9,724-80-9391,BU1111,17.925


In [87]:
query3 = '''
WITH 
    AUTHOR_ROYALTY
AS
(
    SELECT   
        titleauthor.au_id AS AUTHOR_ID,
        titleauthor.title_id AS TITLE_ID,
        titles.price * sales.qty * titles.royalty / 100 * titleauthor.royaltyper / 100 AS ROYALTIES,
        titles.advance * titleauthor.royaltyper / 100 AS ADVANCE
    FROM
        `ironhack-data-analytics-265219.publications.titleauthor` titleauthor
    JOIN
        `ironhack-data-analytics-265219.publications.sales` sales
    ON
        sales.title_id = titleauthor.title_id
    JOIN
        `ironhack-data-analytics-265219.publications.titles` titles
    ON
        titleauthor.title_id = titles.title_id
)
SELECT
    AUTHOR_ID,
    AUTHOR_ROYALTY.TITLE_ID,
    SUM(AUTHOR_ROYALTY.ADVANCE + AUTHOR_ROYALTY.ROYALTIES) AS PROFIT
FROM
    AUTHOR_ROYALTY
GROUP BY
    1,2
ORDER BY 
    SUM(AUTHOR_ROYALTY.ADVANCE + AUTHOR_ROYALTY.ROYALTIES) DESC
LIMIT 3
'''

In [88]:
query_job = client.query(query=query3)
df=query_job.to_dataframe()
df

,AUTHOR_ID,TITLE_ID,PROFIT
0,722-51-5454,MC3021,22521.528
1,213-46-8915,BU2075,10150.116
2,899-46-2035,MC3021,7507.176


In [97]:
sql='''
CREATE TABLE
    `ironhack-data-analytics-265219.publications.most_profiting_authors9`
AS
(
WITH 
    AUTHOR_ROYALTY
AS
(
    SELECT   
        titleauthor.au_id AS AUTHOR_ID,
        titleauthor.title_id AS TITLE_ID,
        titles.price * sales.qty * titles.royalty / 100 * titleauthor.royaltyper / 100 AS ROYALTIES,
        titles.advance * titleauthor.royaltyper / 100 AS ADVANCE
    FROM
        `ironhack-data-analytics-265219.publications.titleauthor` titleauthor
    JOIN
        `ironhack-data-analytics-265219.publications.sales` sales
    ON
        sales.title_id = titleauthor.title_id
    JOIN
        `ironhack-data-analytics-265219.publications.titles` titles
    ON
        titleauthor.title_id = titles.title_id
)
SELECT
    AUTHOR_ID,
    AUTHOR_ROYALTY.TITLE_ID,
    SUM(AUTHOR_ROYALTY.ADVANCE + AUTHOR_ROYALTY.ROYALTIES) AS PROFIT
FROM
    AUTHOR_ROYALTY
GROUP BY
    1,2
ORDER BY 
    SUM(AUTHOR_ROYALTY.ADVANCE + AUTHOR_ROYALTY.ROYALTIES) DESC
LIMIT 3
)
'''

In [96]:
query_job = client.query(query=sql)